In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)
import numpy as np
from ordered_set import OrderedSet
from sklearn.metrics.pairwise import cosine_similarity
import requests

# Data Prep

## SDG Indicators

Source: https://unstats.un.org/sdgs/indicators/indicators-list/

In [2]:
# # Ingest + Prep SDG Corpus
# SDGS = pd.read_excel('sdg_indicators.xlsx')
# SDGS['SDG'] = SDGS['Target'].apply(lambda x: x if str(x).startswith('Goal') else np.nan).ffill()
# SDGS['Target'] = SDGS['Target'].ffill()
# SDGS = SDGS[~SDGS.Target.apply(lambda x: x.startswith('Goal'))]
# SDGS = SDGS.dropna().drop_duplicates().reset_index(drop = True)
# SDGS.insert(0, 'Target No.', SDGS.Target.apply(lambda target: target.split()[0]))
# SDGS.insert(0, 'SDG No.', SDGS.SDG.apply(lambda goal: int(goal.split('.')[0].replace('Goal', '').strip())))
# SDGS.to_csv('sdg_indicators_prepped.csv', index = False)


SDGS = pd.read_csv('sdg_indicators_prepped.csv')

"""
SDG text aggregation setup
"""
SDGS = SDGS.groupby(['SDG No.', 'Target No.', 'SDG', 'Target'])['Indicator'].apply(lambda x: '\n'.join(x)).reset_index().sort_values(by = ['SDG No.', 'SDG']).reset_index(drop = True).rename(columns = {'Indicator' : 'Indicators'})

SDGS.head()

,SDG No.,Target No.,SDG,Target,Indicators
0,1,1.1,Goal 1. End poverty in all its forms everywhere,"1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day","1.1.1 Proportion of the population living below the international poverty line by sex, age, employment status and geographic location (urban/rural)"
1,1,1.2,Goal 1. End poverty in all its forms everywhere,"1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to na...","1.2.1 Proportion of population living below the national poverty line, by sex and age\n1.2.2 Proportion of men, women and children of all ages liv..."
2,1,1.3,Goal 1. End poverty in all its forms everywhere,"1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of...","1.3.1 Proportion of population covered by social protection floors/systems, by sex, distinguishing children, unemployed persons, older persons, pe..."
3,1,1.4,Goal 1. End poverty in all its forms everywhere,"1.4 By 2030, ensure that all men and women, in particular the poor and the vulnerable, have equal rights to economic resources, as well as access ...",1.4.1 Proportion of population living in households with access to basic services\n1.4.2 Proportion of total adult population with secure tenure r...
4,1,1.5,Goal 1. End poverty in all its forms everywhere,"1.5 By 2030, build the resilience of the poor and those in vulnerable situations and reduce their exposure and vulnerability to climate-related ex...","1.5.1 Number of deaths, missing persons and directly affected persons attributed to disasters per 100,000 population\n1.5.2 Direct economic loss a..."


## Legiscan

https://legiscan.com/gaits/documentation/legiscan




Note on statuses:
- 1 = Introduced
- 2 = Held
- 3 = Sent to Executive Branch
- 4 = Passed
- 5 = Vetoed
- 6 = Died

Probably will want to just look at those passed.


In [3]:
def get_bills(apikey, legislative_bodies):

    # Iterate states
    BILLS = []
    for stateid in legislative_bodies:

        try:
            print(f'Fetching {stateid}...')

            # Fetch bill data for state
            url = 'https://api.legiscan.com/?key={}&op=getMasterList&state={}'
            response = requests.get(url.format(apikey, stateid))
            response = response.json()['masterlist']

            # Iterate bills and add rows to dataframe
            for i in range(len(response) - 1):
                bill = response[str(i)]
                BILLS.append([stateid, bill['number'], bill['status'], bill['status_date'], bill['title'], bill['description'], bill['url'], bill['last_action']])
        except:
            print(f'Could not fetch {stateid}')

    # Add bills to dataframe
    BILLS = pd.DataFrame(columns = ['legislative_body', 'bill_number', 'status', 'status_date', 'title', 'description', 'legiscan_url', 'last_action'], data = BILLS).fillna('')
    BILLS['status_date'] = pd.to_datetime(BILLS['status_date'])

    # Map status codes & filter to passed / signed legislation
    statuses = {1 : 'Introduced', 2 : 'Held', 3 : 'Passed', 4 : 'Signed', 5 : 'Vetoed', 6 : 'Died'}
    BILLS['status'] = BILLS.status.apply(lambda x: statuses[x] if x in statuses.keys() else 'Other')

    # Adding a UUID collumn
    BILLS.insert(0, 'UUID', BILLS.apply(lambda row: '-'.join([row['legislative_body'], row['bill_number']]), axis = 1))

    return BILLS

apikey = '3450b965641ca5cf68d314cc3e375ad0'


legislative_bodies = state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY",
    "DC", "US"
]


"""
TEMP - US and LA only
"""

# legislative_bodies = ['US', 'LA']

# BILLS = get_bills(apikey, legislative_bodies)
# BILLS.to_csv('BILLS.csv', index = False)

BILLS = pd.read_csv('BILLS.csv')



# Embedding setup

In [17]:
"""
Generate embeddings for SDGS and Bill titles / descriptions
"""

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')

SDG_embeddings = model.encode(SDGS.apply(lambda row: '\n'.join(list(OrderedSet([row['Target'].strip(), row['Indicators'].strip()]))), axis = 1))
SDGS['embedding'] = list(SDG_embeddings)

BILL_embeddings = model.encode(BILLS.apply(lambda row: '\n'.join(list(OrderedSet([row['title'].strip(), row['description'].strip()]))), axis = 1))
BILLS['embedding'] = list(BILL_embeddings)


# Tagging

In [76]:
def get_sdg_tags(bill_embedding, SDGS = SDGS, threshold = 0.25):

    # Copy SDGS table & calculate similarities
    SDGS_temp = SDGS.copy()
    SDGS_temp['similarity'] = SDGS_temp.embedding.apply(lambda sdg_embedding: 
                                                        cosine_similarity([bill_embedding], [sdg_embedding])[0][0]
                                                        )
    
    # Filter out below-treshold similarity & return top 3 matched targets
    SDGS_temp = SDGS_temp[SDGS_temp.similarity >= threshold].sort_values(by = 'similarity', ascending=False).head(3).reset_index(drop = True)

    # Match payload
    SDG_tags = {'sdgs': list(set(SDGS_temp['SDG No.'])), 'targets': list(set(SDGS_temp['Target No.']))}
    return SDG_tags


def breakout_sdg_tags(BILLS):

    BILLS_TEMP = pd.DataFrame(columns = list(BILLS.drop(columns = 'sdg_tags').columns) + ['sdg_num', 'sdg_name', 'target_num', 'target_name'])
    sdg_dict = SDGS[['SDG No.', 'SDG']].drop_duplicates().set_index('SDG No.')['SDG'].to_dict()
    target_dict = SDGS[['Target No.', 'Target']].drop_duplicates().set_index('Target No.')['Target'].to_dict()

    for row in BILLS.itertuples():
        if len(row.sdg_tags['sdgs']) == 0:
            new_row = [row.UUID, row.legislative_body, row.bill_number, row.status, row.status_date, row.title, row.description, row.legiscan_url, row.last_action, None, None, None, None]
            BILLS_TEMP.loc[len(BILLS_TEMP)] = new_row
        else:
            for sdg in row.sdg_tags['sdgs']:
                for target in row.sdg_tags['targets']:
                    if target.startswith(str(sdg)):
                        new_row = [row.UUID, row.legislative_body, row.bill_number, row.status, row.status_date, row.title, row.description, row.legiscan_url, row.last_action, sdg, sdg_dict[sdg], target, target_dict[target]]
                        BILLS_TEMP.loc[len(BILLS_TEMP)] = new_row

    return BILLS_TEMP


# # Apply tags
# BILLS['sdg_tags'] = BILLS.embedding.apply(lambda embedding: get_sdg_tags(embedding))

# # Breakout tags
# BILLS = breakout_sdg_tags(BILLS)

# # Save to CSV
# BILLS.to_csv('BILLS_TAGGED.csv', index = False)
